# Fields, Semantics, and Probabilistic Results

In [1]:
import cicada.encoder
encoder = cicada.encoder.FixedFieldEncoder(modulus=251, precision=3)

encoder.modulus

251

In [2]:
encoder.fieldbits

8

To represent negative numbers, the encoder defines the `_posbound` attribute, which is the threshold in the field above which values are interpreted as negative.  It is equal to the floor of the modulus halved.  In our example, `_posbound` will be 125, meaning all field values in the range [126,250] will be interpreted as negative numbers. Thus, the field is split into two regions with the upper half dedicated to representing negative numbers while the lower half represents positive numbers:

In [3]:
encoder._posbound

125

In this example, we reserve 3 bits for fractional precision (i.e. the number of bits to the right of the radix), which is the `precision` argument provided when creating the encoder.  Any fractional precision requiring more than 3 bits in the value to be encoded will be lost via the encoding process.

When encoding and decoding fractional values, their bits are shifted left or right respectively, to map them to integers in the field.   Conceptually, this can be thought of as multiplying or dividing by a scale value that is equal to $2^{precision}$, or $2^3=8$ in this case:

In [4]:
encoder.precision

3

In [5]:
encoder._scale

8

The mapping from fractional values to positive integers in the field have many follow-on implications. The first is that overflow and underflow can and will happen without any notice if code is not written with the field size in mind, since all operations occur obliviously.  This may happen in unexpected situations and yield similarly unexpected results. For example, the addition of two positive values may yield a negative seemingly non-sensical result if their sum puts them into the upper half of the field which will later be decoded as a negative value.  Ensure that field sizes are large enough to make this impossible (or at least unlikely). 

Secondly, division is not directly possible in the context of the field since it is an integral field and no notion of values less than one exist in that context. We use field elements to represent fractional values, but these are semantics that have no significance to the field itself. We can get a stable and expected result for division by multiplying with an element's multiplicative inverse in the context of the field, but this has the desired result if-and-only-if the intended dividend has the desired divisor as a factor. Otherwise the result will not yield any useful value for the external semantics. In general, we perform division via approximation, masking, and the like. The accuracy of the result from any division operation is heavily dependent on the precision available from the encoder with respect to the number of bits right of the radix.  

Let's try some examples using the parameters and encoder described above. For each of the following we will provide the example, work it out "by hand" and then show what it looks like in Cicada. 

**Encode and decode the value 3.25**

* We multiply by the scale (in this case $2^3$) $3.25\cdot8=26$. This is positive and less than the modulus so there are no concerns here; we are done.
* To decode we check if the value (26) is greater than `_posbound` (it isn’t) so we divide by the scale and return the value $26/8=3.25$

In [6]:
import numpy

value = numpy.array(3.25)
print(f"        Value: {value}")

encoded = encoder.encode(value)
print(f"Encoded Value: {encoded}")

decoded = encoder.decode(encoded)
print(f"Decoded Value: {decoded}")

        Value: 3.25
Encoded Value: 26
Decoded Value: 3.25


**Encode and decode the value -3.25**

* We multiply by the scale (in this case $2^3$) $-3.25\cdot8=-26$. This is negative so we apply the modulus i.e., $-26 \mod{251}=225$.
* To decode we check if the value (225) is greater than posbound (it is) so we compute the additive inverse of the difference between the modulus and the value i.e., $-(251-225)=-26$, then divide by the scale and return the value $-26/8=-3.25$.


In [7]:
value = numpy.array(-3.25)
print(f"        Value: {value}")

encoded = encoder.encode(value)
print(f"Encoded Value: {encoded}")

decoded = encoder.decode(encoded)
print(f"Decoded Value: {decoded}")

        Value: -3.25
Encoded Value: 225
Decoded Value: -3.25


**Encode and decode the value 3.0625**

* We multiply by the scale $3.0625*8=24.5$ This is positive and less than the modulus, but not an integral value so we truncate to 24. We are done.
* To decode we check if the value (24) is greater than posbound (it isn’t) so we divide by the scale and return the value $24/8=3$
* Checking against the original value it is clear to see that we have lost the fractional part of the original (0.0625). This is due to the fact that in binary it is represented as 0.0001 and we have only 3 bits of binary precision available. Specifically, this happened at the point we truncated 24.5 to 24 which is a necessary step to make sure every value is both consistent in semantics and compatible with representation in our integral field.

In [8]:
value = numpy.array(3.0625)
print(f"        Value: {value}")

encoded = encoder.encode(value)
print(f"Encoded Value: {encoded}")

decoded = encoder.decode(encoded)
print(f"Decoded Value: {decoded}")

        Value: 3.0625
Encoded Value: 24
Decoded Value: 3.0


**Encode, add, and decode 15 and 2**

* In a similar manner to the preceding, the encoding of 15 and 2 is 120 and 16 respectively. 
* The sum of these is 136
* Decoding 136 yields -14.375, not the answer we were expecting as the sum of 15 and 2, due to overflow of the representable positive range in our semantic mapping onto the field. In practice much larger fields are used so that incidents such as this are far easier to avoid. For example, a 64 bit field is used in Cicada by default, and you are free to create larger fields, within practical limits.

In [9]:
value1 = numpy.array(15)
print(f"        Value1: {value1}")

value2 = numpy.array(2)
print(f"        Value2: {value2}")

encoded1 = encoder.encode(value1)
print(f"Encoded Value1: {encoded1}")

encoded2 = encoder.encode(value2)
print(f"Encoded Value2: {encoded2}")

encoded_sum = encoder.add(encoded1, encoded2)
print(f"   Encoded Sum: {encoded_sum}")
      
decoded_sum = encoder.decode(encoded_sum)
print(f"   Decoded Sum: {decoded_sum}")

        Value1: 15
        Value2: 2
Encoded Value1: 120
Encoded Value2: 16
   Encoded Sum: 136
   Decoded Sum: -14.375


**Min and Max**

Another area of concern wrt such issues are the min and max functions. Given the semantic meaning we are mapping onto the field, problems may arise at the border. Our implementation of these functions is based on the following algebraic expressions:

$$min(x, y)=(x+y+abs(x-y))/2$$

$$max(x, y)=(x+y+abs(x-y))/2$$

This will behave as expected much of the time; however, if the difference between $x$ and $y$ wraps around an end of the field more than once then problems can occur. Given a field $\mathbb{Z}_p$, as long as both operands are of the same sign or both satisfy the (in our opinion reasonable) constraint that $abs(x)<p//4$, then the min and max functions should behave as anticipated.

**Probabilistic Results**

There are some functions we have implemented which may be off from the true answer by some relatively negligible margin. For example, consider the approach we have implemented for truncation. The function masks a shared secret with a random field element. It is possible that the sum of the operand and the random field element will overflow the field, which would cause the whole process to be wildly inaccurate. This is very unlikely to happen in practice, so long as the operands are relatively small in comparison to the size of the field. Additionally, a small error (overflow in the least significant bits generated from a carry coming into this region from the bits that were truncated) is also a possibility. For example:

In [12]:
import logging

import cicada.additive
import cicada.logging
from cicada.communicator import SocketCommunicator

logging.basicConfig(level=logging.INFO)

def main(communicator):
    log = cicada.Logger(logging.getLogger(), communicator)
    protocol = cicada.additive.AdditiveProtocol(communicator)#, None, None, 251, 3)
    
    test_val = numpy.array(65536.5)
    shared_val = protocol.share(src=0, secret=protocol.encoder.encode(test_val), shape=test_val.shape)
    truncd_val = 1.0000152587890625
    counter = 0
    right_count = 0
    err_sum = 0
    while counter <100:     
        truncd_shared_val = protocol.truncate(shared_val)    
        truncd_val = protocol.encoder.decode(protocol.reveal(truncd_shared_val))
        counter += 1
        #log.info(f'attempt {counter}', src=0)
        if truncd_val == 1:
            right_count += 1
        else:
            err_sum += abs(truncd_val-1)
    log.info(f"Player {communicator.rank} \n\tTest Value: \t\t{test_val}\n\tTruncd Value: \t\t{truncd_val}\n\tExpected Value:\t\t{1.0}\n\tPercent Correct: \t{right_count}\n\tAverage Error:\t\t{err_sum/(100-right_count)}\n\tAverage answer:\t\t{1*(right_count/100)+(1+err_sum/(100-right_count))*(100-right_count)/100}", src=1)    
        
SocketCommunicator.run(main, world_size=3);

INFO:cicada.communicator.socket:Comm 'world' player 0 rendezvous with tcp://127.0.0.1:50361 from tcp://127.0.0.1:50361.
INFO:cicada.communicator.socket:Comm 'world' player 1 rendezvous with tcp://127.0.0.1:50361 from tcp://127.0.0.1:50362.
INFO:cicada.communicator.socket:Comm 'world' player 2 rendezvous with tcp://127.0.0.1:50361 from tcp://127.0.0.1:50365.
INFO:cicada.communicator.socket:Comm 'world' player 0 communicator ready.
INFO:cicada.communicator.socket:Comm 'world' player 1 communicator ready.
INFO:cicada.communicator.socket:Comm 'world' player 2 communicator ready.
INFO:root:Player 1 
	Test Value: 		65536.5
	Truncd Value: 		1.0
	Expected Value:		1.0
	Percent Correct: 	52
	Average Error:		1.52587890625e-05
	Average answer:		1.00000732421875
INFO:cicada.communicator.socket:Comm 'world' player 0 communicator freed.
INFO:cicada.communicator.socket:Comm 'world' player 1 communicator freed.
INFO:cicada.communicator.socket:Comm 'world' player 2 communicator freed.
INFO:cicada.commun

Here, we perform 100 truncations on a value which is specifically selected to given an error probability of 50%. The `Percent Correct` reported above should be somewhere between 45 and 55, and the average error should be less than ~2e-5. This is caused by specifically setting the most significant bit in the region to be truncated thereby making it a roughly 50% probability based on the underlying random bit generation protocol that the corresponding bit in the mask will be set which in turn generates a cary into the region which will be unaffected by the truncation yielding the relatively insignificant error we see.